In [1]:
!pip install sentence-transformers
!pip install tf-keras
!pip install tensorflow
!pip install sentence-transformers
!pip install seaborn
!pip install nltk

In [2]:
!pip install sentence-transformers
!pip install tf-keras
!pip install tensorflow
!pip install sentence-transformers
!pip install seaborn
!pip install nltk

In [7]:
import string
import re
import nltk
import os
import pandas as pd
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
import pickle

nltk.download("wordnet")
hr_data=pd.read_csv('data_with_internal_info.csv', low_memory=False)

def text_prep(text):
    text = [str(t).lower() for t in text]
    table = str.maketrans('', '', string.punctuation)
    text = [t.translate(table) for t in text]
    text = [t.replace('\n', ' ') for t in text]
    text = [re.sub(r'[^a-zA-Z0-9._\s]', '', t) for t in text]
    text = [re.sub(r'\s+', ' ', t).strip() for t in text]
    return text
os.environ["LD_LIBRARY_PATH"] = os.path.join(os.environ["CONDA_PREFIX"], "lib") + ":" + os.environ.get("LD_LIBRARY_PATH", "")
hr_data['Yearly Gross Base Salary'] = pd.to_numeric(hr_data['Yearly Gross Base Salary'], errors='coerce')
hr_data = hr_data.dropna(subset=['Yearly Gross Base Salary'])
columns_to_combine = [
    'Job Title (en)', 'Job Title (nl)', 'Level Title', 'Function goal', 
    'Key result areas: result area (1)', 'Key result areas: result area (2)',
    'Key result areas: result area (3)', 'Key result areas: result area (4)',
    'Key result areas: result area (5)', 'Key result areas: result area (6)',
    'Key result areas: result area (7)', 'Key result areas: result area (8)',
    'Key result areas: result area (9)', 'Key result areas: result area (10)',
    'Key result areas: result area (11)', 'Specify the budget amounts.', 
    'Diploma Category', 'Speciality', 'Required experience', 'Innovation', 
    'Row 4 - Column 1', 'Row 5 - Column 1',
    'Row 6 - Column 1', 'Row 7 - Column 1', 'Row 8 - Column 1','Internal Job', 'Internal Job Grade', 'Department']

hr_data[columns_to_combine] = hr_data[columns_to_combine].astype(str)
text_combined = hr_data[columns_to_combine].agg(' '.join, axis=1).tolist()
text_cleaned = text_prep(text_combined)

model_Sen = SentenceTransformer('paraphrase-MiniLM-L12-v2')
text_cleaned = text_prep(text_combined)

embeddings = model_Sen.encode(text_cleaned, convert_to_numpy=True, show_progress_bar=True, batch_size=100)

X = pd.DataFrame(embeddings)

y = hr_data[['Yearly Gross Base Salary', 'Compas Grade']].copy()
y_imputed = y.fillna(y.mean())

scaler_y = StandardScaler()
y_scaled = pd.DataFrame(scaler_y.fit_transform(y_imputed), columns=y_imputed.columns)


X_train, X_test, y_train, y_test = train_test_split(X, y_scaled, test_size=0.2, random_state=11)


model = Sequential([
    Input(shape=(X_train.shape[1],)),  #i have 1 input with 384 feature (minilm) and 2 output
    Dense(512, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(2)  
])

model.compile(loss='mse', optimizer='adam', metrics=['mae', 'mse'])
model.summary()

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(35).repeat()
validation_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(35)

model.fit(train_dataset, epochs=40, steps_per_epoch=len(X_train) // 35,
          validation_data=validation_dataset, validation_steps=len(X_test) // 35)

y_pred_scaled = model.predict(X_test)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_true = scaler_y.inverse_transform(y_test)


print("--- Separate Target Evaluation ---")
for i, col in enumerate(y.columns):
    print(f"{col}:")
    print("  MAE:", mean_absolute_error(y_true[:, i], y_pred[:, i]))
    print("  RMSE:", mean_squared_error(y_true[:, i], y_pred[:, i]) ** 0.5)
    print("  R2:", r2_score(y_true[:, i], y_pred[:, i]))
    print("  MAPE:", mean_absolute_percentage_error(y_true[:, i], y_pred[:, i]))
    
model.save("salary_prediction_model.keras")

[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Epoch 1/40


I0000 00:00:1762524508.084778   59882 service.cc:152] XLA service 0x7f6d7000d490 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1762524508.084822   59882 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2025-11-07 14:08:28.152153: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1762524508.539363   59882 cuda_dnn.cc:529] Loaded cuDNN version 91002


 42/287 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.7253 - mae: 0.6499 - mse: 0.7253

I0000 00:00:1762524512.128181   59882 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


287/287 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - loss: 0.3794 - mae: 0.4507 - mse: 0.3794 - val_loss: 0.2977 - val_mae: 0.3825 - val_mse: 0.2977
Epoch 2/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.3044 - mae: 0.3971 - mse: 0.3044 - val_loss: 0.2933 - val_mae: 0.3741 - val_mse: 0.2933
Epoch 3/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2873 - mae: 0.3827 - mse: 0.2873 - val_loss: 0.2747 - val_mae: 0.3578 - val_mse: 0.2747
Epoch 4/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2778 - mae: 0.3760 - mse: 0.2778 - val_loss: 0.2701 - val_mae: 0.3484 - val_mse: 0.2701
Epoch 5/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2750 - mae: 0.3716 - mse: 0.2750 - val_loss: 0.2717 - val_mae: 0.3635 - val_mse: 0.2717
Epoch 6/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2705 - mae: 0.3659 - mse: 0.2705 - val_loss: 0.2645 - val_mae: 0.3508 - val_mse: 0.2645
Epoch 7/40
287/287 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.2664 - mae: 0.3635 - mse: 0.2664 - val_loss: 0.27